In [1]:
import mindspore
import argparse
import numpy as np
import logging
import mindspore.dataset as ds
import os

import json

from tqdm import tqdm
from datetime import datetime
from mindspore.nn import CrossEntropyLoss
from mindspore import nn, ops
from mindspore.train.serialization import save_checkpoint
from mindspore.dataset import TextFileDataset

from mindnlp.transforms import BertTokenizer
from mindnlp.modules import Accumulator
from mindnlp.models import GPT2Config, GPT2LMHeadModel

[ERROR] ME(44611:140086138365760,MainProcess):2023-05-10-14:56:57.339.080 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcu*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
[ERROR] ME(44611:140086138365760,MainProcess):2023-05-10-14:56:57.363.851 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcudnn*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
/data/miniconda3/envs/mindspore/lib/python3.7/site-packages/mindnlp/utils/download.py:26: Tqd

In [2]:
from mindnlp.utils import cache_file

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path, _ = cache_file('train_with_summ.txt', './', url)

In [3]:
dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

In [4]:
train_dataset, eval_dataset, test_dataset = dataset.split([0.8, 0.1, 0.1])

In [5]:
# article: [CLS] xxxxx [SEP]
# summary: [CLS] xxxxx [SEP]

In [6]:
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=32, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        article_len = len(article)
        summary_len = len(summary)

        sep_id = np.array([tokenizer.sep_token_id])
        pad_id = np.array([tokenizer.pad_token_id])
        if article_len + summary_len > max_seq_len:
            new_article_len = max_seq_len - summary_len
            merged = np.concatenate([article[:new_article_len], sep_id, summary[1:]])
        elif article_len + summary_len - 1 < max_seq_len:
            pad_len = max_seq_len - article_len - summary_len + 1
            pad_text = np.array([tokenizer.pad_token_id] * pad_len)
            merged = np.concatenate([article, summary[1:], pad_text])
        else:
            merged = np.concatenate([article, summary[1:]])
        
        labels = np.concatenate([merged[1:], pad_id])
            
        return merged, labels

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(tokenizer, 'article')
    dataset = dataset.map(tokenizer, 'summary')
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])
    
    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [8]:
train_dataset = process_dataset(train_dataset, tokenizer)
eval_dataset = process_dataset(eval_dataset, tokenizer)
test_dataset = process_dataset(test_dataset, tokenizer)

In [9]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[32, 1024], dtype=Int64, value=
 [[ 101, 8119,  118 ...    0,    0,    0],
  [ 101, 2190, 3683 ...    0,    0,    0],
  [ 101, 3862, 1366 ...    0,    0,    0],
  ...
  [ 101, 1355, 2357 ...    0,    0,    0],
  [ 101, 1298, 3736 ... 7564, 6356,  102],
  [ 101, 5468, 5143 ... 4895, 2042,  102]]),
 Tensor(shape=[32, 1024], dtype=Int64, value=
 [[8119,  118, 8140 ...    0,    0,    0],
  [2190, 3683,  671 ...    0,    0,    0],
  [3862, 1366, 1159 ...    0,    0,    0],
  ...
  [1355, 2357, 3189 ...    0,    0,    0],
  [1298, 3736, 1344 ... 6356,  102,    0],
  [5468, 5143,  100 ... 2042,  102,    0]])]

In [10]:
len(tokenizer)

21128

In [18]:
epochs = 6
batch_size = 8

lr = 1e-4
warmup_steps = 2000
accumulate_step = 2
max_grad_norm = 1.0

log_step = 100

In [15]:
config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2LMHeadModel(config, ignore_index=tokenizer.pad_token_id)
optimizer = nn.AdamWeightDecay(model.trainable_params(), lr)
accumulator = Accumulator(optimizer, accumulate_step, max_grad_norm)

In [16]:
# Define forward function
def forward_fn(input_ids, labels):
    logits = model(input_ids, labels=labels)
    return loss / accumulate_step

# Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

# Define function of one-step training
# @mindspore.jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    accumulator(grads)
    return loss

In [17]:
# 记录模型参数数量
num_parameters = 0
parameters = model.trainable_params()
for parameter in parameters:
    num_parameters += parameter.numel()
print('number of model parameters: {}'.format(num_parameters))

number of model parameters: 118295040


In [ ]:
for epoch in range(epochs):
    for batch_idx, (input_ids, labels) in enumerate(train_dataset.create_tuple_iterator()):
        loss = train_step(input_ids, labels)

        if batch_idx % log_step == 0:
            loss, current = loss.asnumpy(), batch_idx
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")